<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo11n_Shikibo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.4 MB/s eta 0:00:00


In [6]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

sending incremental file list
dataset.zip
          1.19G 100%   49.57MB/s    0:00:22 (xfr#1, to-chk=0/1)
Archive:  /content/dataset.zip
   creating: /content/dataset/
  inflating: /content/dataset/data.yaml  
   creating: /content/dataset/images/
  inflating: /content/dataset/images/CAM1_250801135246.jpg  
  inflating: /content/dataset/images/CAM1_250804080357.jpg  
  inflating: /content/dataset/images/CAM1_250804114157.jpg  
  inflating: /content/dataset/images/CAM1_250805024725.jpg  
  inflating: /content/dataset/images/CAM1_250805050017.jpg  
  inflating: /content/dataset/images/CAM1_250805133915.jpg  
  inflating: /content/dataset/images/CAM1_250806104530.jpg  
  inflating: /content/dataset/images/CAM1_250806104734.jpg  
  inflating: /content/dataset/images/CAM1_250806111847.jpg  
  inflating: /content/dataset/images/CAM1_250806131946.jpg  
  inflating: /content/dataset/images/CAM1_250807090505.jpg  
  inflating: /content/dataset/images/CAM1_250820133007.jpg  
  inflating: /conten

In [7]:
model_path = "/content/drive/MyDrive/yolo11n.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"
# 学習/データのパス
model_path = "/content/drive/MyDrive/yolo11n.pt"              # ←必要なら変更

In [8]:
import yaml

# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [9]:
import numpy as np
import os
from ultralytics import YOLO

def train_yolo(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    print(f"Training YOLO11 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,            # 300
        batch=batch_size,         # 32
        imgsz=img_size,           # 960
        project=output_dir,       # "runs_experiment"
        name="yolo11_training",

        # 既存
        iou=iou_threshold,        # 0.5
        mosaic=mosaic,            # 0.0
        mixup=mixup,              # 0.0
        degrees=4,

        # 下で定義済みのパラメータ（有効化）
        lr0=lr0,                  # 0.003
        weight_decay=weight_decay,# 0.001
        label_smoothing=label_smoothing,  # 0.05（非推奨Warningは出ます／外しても可）
        copy_paste=copy_paste,    # 0.05
        translate=translate,      # 0.05
        close_mosaic=close_mosaic,# 10
        perspective=perspective,  # 0.0
        shear=shear,              # 0.0
        fliplr=fliplr,            # 0.1
        flipud=flipud,            # 0.0
        rect=rect,                # True
        cos_lr=cos_lr,            # True
        multi_scale=multi_scale,  # True
        patience=patience,        # 40
        seed=seed,                # 0
        optimizer="AdamW"         # AdamW はサポートされています
    )
    print(f"Training complete. Results saved to {output_dir}")



# 主要ハイパラ
patience = 0
epochs      = 300
batch_size  = 32
img_size    = 960
output_dir  = "runs_experiment"

# 評価/抑制まわり
iou_threshold = 0.5
mosaic       = 0.0
mixup        = 0.0

# train_yolo() 内で参照されるハイパラ（関数に渡さず“変数”として使われます）
lr0              = 0.003
weight_decay     = 0.001
label_smoothing  = 0.05   # ←警告が出るのが鬱陶しければ 0.0 にしてください
copy_paste       = 0.05
translate        = 0.05
close_mosaic     = 10
perspective      = 0.0
shear            = 0.0
fliplr           = 0.1
flipud           = 0.0
rect             = True
cos_lr           = True
multi_scale      = True
patience         = 40
seed             = 0

# 出力先作成
import os
os.makedirs(output_dir, exist_ok=True)

# ==== 学習実行（train_yolo は既に定義済みである前提）====
train_yolo(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading model from /content/drive/MyDrive/yolo11n.pt
Training YOLO11 with 300 epochs...
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset/data.yaml, degrees=4, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.

In [10]:
# === Stage 2: 高解像度(1280)で短期微調整 ===
# 1回だけ実行するガード
if 'STAGE2_ONCE' in globals() and STAGE2_ONCE:
    print("Stage-2 already executed in this runtime. Skip.")
    raise SystemExit
STAGE2_ONCE = True

import os, time

def find_latest_best(project_dir, run_name_contains="yolo11_training"):
    latest = None
    latest_mtime = -1
    for root, dirs, files in os.walk(project_dir):
        if root.endswith("weights") and "best.pt" in files and run_name_contains in root:
            p = os.path.join(root, "best.pt")
            m = os.path.getmtime(p)
            if m > latest_mtime:
                latest = p
                latest_mtime = m
    if latest is None:
        cand = os.path.join(project_dir, "yolo11_training", "weights", "best.pt")
        if os.path.exists(cand):
            return cand
        raise FileNotFoundError(f"No best.pt found under {project_dir}")
    return latest

stage2_model_path = find_latest_best(output_dir)
print("Stage2 base model:", stage2_model_path)

img_size_stage2   = 1280
epochs_stage2     = 50
batch_size_stage2 = max(8, int(batch_size * (img_size / img_size_stage2) ** 2))
print("batch_size_stage2 =", batch_size_stage2)

# 出力先を分ける（推奨）
output_dir_stage2 = os.path.join(output_dir, "stage2")
os.makedirs(output_dir_stage2, exist_ok=True)

# 一時的に lr0 と multi_scale を上書き（関数は流用）
_lr0_bak = lr0
_multi_bak = multi_scale
try:
    lr0 = 0.001
    multi_scale = False

    train_yolo(
        model_path=stage2_model_path,
        data_yaml=data_yaml,
        epochs=epochs_stage2,
        batch_size=batch_size_stage2,
        img_size=img_size_stage2,
        output_dir=output_dir_stage2,   # ← 別フォルダへ
        iou_threshold=iou_threshold,
        mosaic=0.0,
        mixup=0.0
    )
finally:
    lr0 = _lr0_bak
    multi_scale = _multi_bak



Stage2 base model: runs_experiment/yolo11_training/weights/best.pt
batch_size_stage2 = 18
Loading model from runs_experiment/yolo11_training/weights/best.pt
Training YOLO11 with 50 epochs...
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=18, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset/data.yaml, degrees=4, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.1, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None,

In [11]:
# best.pt のパス
best_model_path = 'runs_experiment/yolo11_training/weights/best.pt'

# best.pt が存在すれば自動でダウンロード
if os.path.exists(best_model_path):
    files.download(best_model_path)
else:
    print("best.pt が見つかりませんでした。")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>